## Prompt Optimization techniques

In [16]:
import requests
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

from openai import OpenAI

In [3]:
requests.get("http://localhost:11434").content

OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

model_name="llama3.2"

In [5]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
#from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.chains import ConversationChain
from langchain_classic.memory import ConversationBufferMemory

In [6]:
# Initialize the Ollama model
llm = ChatOllama(
    model=model_name,   # change to mistral / qwen2.5 if you want
    temperature=0
)

In [10]:
# Define a helper function to generate responses
def generate_response(prompt):
    """Generate a response using the language model.

    Args:
        prompt (str): The input prompt.

    Returns:
        str: The generated response.
    """
    return llm.invoke(prompt).content

## A/B Testing Prompts

In [18]:
# Define prompt variations
prompt_a = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in simple terms."
)

prompt_b = PromptTemplate(
    input_variables=["topic"],
    template="Provide a beginner-friendly explanation of {topic}, including key concepts and an example."
)

# Updated function to evaluate response quality
def evaluate_response(response, criteria):
    """Evaluate the quality of a response based on given criteria.

    Args:
        response (str): The generated response.
        criteria (list): List of criteria to evaluate.

    Returns:
        float: The average score across all criteria.
    """
    scores = []
    for criterion in criteria:
        print(f"Evaluating response based on {criterion}...")
        prompt = f"On a scale of 1-10, rate the following response on {criterion}. Start your response with the numeric score:\n\n{response}"
        response = generate_response(prompt)
        # show 50 characters of the response
        # Use regex to find the first number in the response
        score_match = re.search(r'\d+', response)
        if score_match:
            score = int(score_match.group())
            scores.append(min(score, 10))  # Ensure score is not greater than 10
        else:
            print(f"Warning: Could not extract numeric score for {criterion}. Using default score of 5.")
            scores.append(5)  # Default score if no number is found
    return np.mean(scores)

# Perform A/B test
topic = "machine learning"
response_a = generate_response(prompt_a.format(topic=topic))
response_b = generate_response(prompt_b.format(topic=topic))

criteria = ["clarity", "informativeness", "engagement"]
score_a = evaluate_response(response_a, criteria)
score_b = evaluate_response(response_b, criteria)

print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Prompt A score: 8.00
Prompt B score: 8.00
Winning prompt: B


## Iterative Refinement

In [45]:
import re

def sanitize_template(template: str) -> str:
    """Escape all curly braces that are NOT the {topic} placeholder."""
    # Find all placeholders in the template
    placeholders = re.findall(r'\{(.*?)\}', template)
    for p in placeholders:
        if p != 'topic':
            # Escape by doubling the braces so formatter ignores them
            template = template.replace('{' + p + '}', '{{' + p + '}}')
    return template


def refine_prompt(initial_prompt, topic, iterations=3):
    current_prompt = initial_prompt

    for i in range(iterations):
        # Sanitize before attempting format
        current_prompt.template = sanitize_template(current_prompt.template)

        try:
            response = generate_response(current_prompt.format(topic=topic))
        except KeyError as e:
            print(f"Error in iteration {i+1}: Missing key {e}. Adjusting prompt...")
            # Fallback: escape the offending key and retry
            bad_key = e.args[0]
            current_prompt.template = current_prompt.template.replace(
                '{' + bad_key + '}', '{{' + bad_key + '}}'
            )
            response = generate_response(current_prompt.format(topic=topic))

        # Generate feedback
        feedback_prompt = (
            f"Analyze the following explanation of {topic} and suggest "
            f"improvements to the prompt that generated it:\n\n{response}"
        )
        feedback = generate_response(feedback_prompt)

        # Refine the prompt — explicitly instruct LLM to avoid extra braces
        refine_prompt_text = (
            f"Based on this feedback: '{feedback}', improve the following prompt template.\n"
            f"IMPORTANT: Use ONLY the placeholder {{topic}} in your template. "
            f"Do NOT use any other curly braces or placeholders.\n\n"
            f"{current_prompt.template}"
        )
        refined_template = generate_response(refine_prompt_text)

        # Sanitize the LLM output before creating new PromptTemplate
        refined_template = sanitize_template(refined_template)

        current_prompt = PromptTemplate(
            input_variables=["topic"],
            template=refined_template
        )

        print(f"Iteration {i+1} prompt:\n{current_prompt.template}\n")

    return current_prompt


# Perform A/B test
topic = "machine learning"
response_a = generate_response(prompt_a.format(topic=topic))
response_b = generate_response(prompt_b.format(topic=topic))

criteria = ["clarity", "informativeness", "engagement"]
score_a = evaluate_response(response_a, criteria)
score_b = evaluate_response(response_b, criteria)

print(f"Prompt A score: {score_a:.2f}")
print(f"Prompt B score: {score_b:.2f}")
print(f"Winning prompt: {'A' if score_a > score_b else 'B'}")

# Start with the winning prompt from A/B testing
initial_prompt = prompt_b if score_b > score_a else prompt_a
refined_prompt = refine_prompt(initial_prompt, "machine learning")

print("\nFinal refined prompt:")
print(refined_prompt.template)

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Prompt A score: 8.00
Prompt B score: 8.00
Winning prompt: B
Iteration 1 prompt:
Imagine you're teaching a child to recognize different animals. You show them pictures of dogs, cats, and birds, and say "this is a dog" or "this is a cat". At first, the child might not be very good at recognizing the animals, but as they see more and more pictures, they start to get better.

{topic} works in a similar way. A computer is shown a large number of examples, such as images of dogs and cats, and it tries to figure out the characteristics that make them different. This process involves identifying patterns and relationships between the examples, which allows the computer to learn from its mistakes and improve over time.

There are three m

## Comparing Original and Refined Prompts

In [48]:
original_response = generate_response(initial_prompt.format(topic="machine learning"))
refined_prompt = generate_response(refined_prompt.format(topic="machine learning"))

original_score = evaluate_response(original_response, criteria)
refined_score = evaluate_response(refined_prompt, criteria)

print(f"Original prompt score: {original_score:.2f}")
print(f"Refined prompt score: {refined_score:.2f}")
print(f"Improvement: {(refined_score - original_score):.2f} points") 

Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Evaluating response based on clarity...
Evaluating response based on informativeness...
Evaluating response based on engagement...
Original prompt score: 8.00
Refined prompt score: 8.00
Improvement: 0.00 points


# Handling Ambiguity  and  Improving Clarity in Prompt Engineering

In [52]:
ambiguous_prompts = [
    "Tell me about the bank.",
    "What's the best way to get to school?",
    "Can you explain the theory?"
]

for prompt in ambiguous_prompts:
    analysis_prompt = f"Analyze the following prompt for ambiguity: '{prompt}'. Explain why it's ambiguous and list possible interpretations."
    print(f"Prompt: {prompt}")
    print(llm.invoke(analysis_prompt).content)
    print("-" * 50)

Prompt: Tell me about the bank.
The prompt "Tell me about the bank" is ambiguous because it lacks specificity, leaving room for multiple interpretations. Here are some reasons why:

1. **Lack of clarity**: The word "bank" can refer to a financial institution (e.g., a commercial bank), a riverbank, or even a storage location for data (a database). Without further context, it's unclear which meaning is intended.
2. **Multiple possible meanings**: There are at least three common interpretations of the term "bank":
	* A financial institution providing banking services (e.g., depositing, lending, and investing).
	* The side or edge of a river or lake.
	* A storage location for data or information.
3. **Contextual dependence**: The interpretation of "bank" depends on the context in which it is used. For example:
	* In a financial setting, "Tell me about the bank" might refer to a specific commercial bank or a banking industry as a whole.
	* In a geographical setting, "Tell me about the bank"

## Resolving Ambiguity

In [55]:
def resolve_ambiguity(prompt, context):
    """
    Resolve ambiguity in a prompt by providing additional context.
    
    Args:
    prompt (str): The original ambiguous prompt
    context (str): Additional context to resolve ambiguity
    
    Returns:
    str: The AI's response to the clarified prompt
    """
    clarified_prompt = f"{context}\n\nBased on this context, {prompt}"
    return llm.invoke(clarified_prompt).content

# Example usage
ambiguous_prompt = "Tell me about the bank."
contexts = [
    "You are a financial advisor discussing savings accounts.",
    "You are a geographer describing river formations."
]

for context in contexts:
    print(f"Context: {context}")
    print(f"Clarified response: {resolve_ambiguity(ambiguous_prompt, context)}")
    print("-" * 50)

Context: You are a financial advisor discussing savings accounts.
Clarified response: As a financial advisor, I'd be happy to discuss savings accounts with you.

When it comes to choosing a bank for your savings account, there are several factors to consider. Here are some key things to look for:

1. **Interest Rates**: Look for banks that offer competitive interest rates on their savings accounts. While rates can fluctuate over time, a higher rate can help your money grow faster.
2. **Fees**: Be aware of any fees associated with the account, such as maintenance fees, overdraft fees, or ATM fees. Some banks may charge these fees, while others may not.
3. **Minimum Balance Requirements**: Some savings accounts require you to maintain a minimum balance to avoid fees or earn interest. Make sure you understand the requirements and can meet them easily.
4. **Accessibility**: Consider the bank's online and mobile banking capabilities. Can you access your account easily from anywhere? Are the

## Techniques for Writing Clearer Prompts

In [58]:
def compare_prompt_clarity(original_prompt, improved_prompt):
    """
    Compare the responses to an original prompt and an improved, clearer version.
    
    Args:
    original_prompt (str): The original, potentially unclear prompt
    improved_prompt (str): An improved, clearer version of the prompt
    
    Returns:
    tuple: Responses to the original and improved prompts
    """
    original_response = llm.invoke(original_prompt).content
    improved_response = llm.invoke(improved_prompt).content
    return original_response, improved_response

# Example usage
original_prompt = "How do I make it?"
improved_prompt = "Provide a step-by-step guide for making a classic margherita pizza, including ingredients and cooking instructions."

original_response, improved_response = compare_prompt_clarity(original_prompt, improved_prompt)

print("Original Prompt Response:")
print(original_response)
print("\nImproved Prompt Response:")
print(improved_response)

Original Prompt Response:
I'm happy to help, but I need a bit more information. You didn't specify what "it" is that you want to make. Could you please provide more context or clarify what you're trying to create? Is it a recipe, a craft project, a piece of art, or something else entirely? I'll do my best to assist you once I have a better understanding of what you're looking for.

Improved Prompt Response:
Here's a step-by-step guide to making a classic Margherita pizza:

Ingredients:

For the dough:

* 1 cup warm water
* 2 teaspoons active dry yeast
* 3 tablespoons olive oil
* 1 teaspoon salt
* 4 cups all-purpose flour

For the sauce:

* 2 cups crushed San Marzano tomatoes (or other fresh, flavorful tomatoes)
* 2 cloves garlic, minced
* 1 tablespoon olive oil
* Salt and pepper to taste

For the toppings:

* 8 ounces fresh mozzarella cheese, sliced into thin rounds
* Fresh basil leaves

Instructions:

**Step 1: Make the Dough**

1. In a large mixing bowl, combine the warm water and ye

## Structured Prompts for Clarity

In [61]:
structured_prompt = PromptTemplate(
    input_variables=["topic", "aspects", "tone"],
    template="""Provide an analysis of {topic} considering the following aspects:
    1. {{aspects[0]}}
    2. {{aspects[1]}}
    3. {{aspects[2]}}
    
    Present the analysis in a {tone} tone.
    """
)

# Example usage
input_variables = {
    "topic": "the impact of social media on society",
    "aspects": ["communication patterns", "mental health", "information spread"],
    "tone": "balanced and objective"
}

chain = structured_prompt | llm
response = chain.invoke(input_variables).content
print(response)

Based on available research and data, here's an analysis of the impact of social media on society across three key aspects:

**Aspect 1: Social Connections and Community Building**

Social media has revolutionized the way people connect with each other. Platforms like Facebook, Instagram, and Twitter have enabled users to maintain relationships with friends and family who live far away, as well as connect with new people who share similar interests. This has led to a sense of community and belonging among social media users.

On the positive side, social media has:

* Enabled people to stay in touch with loved ones who are geographically distant
* Provided opportunities for people to connect with others who share similar interests or passions
* Facilitated the growth of online communities around shared causes or hobbies

However, on the negative side, excessive social media use can lead to feelings of loneliness and isolation, as well as decreased face-to-face interaction skills.

**As

## Improving Prompt Clarity

In [64]:
unclear_prompts = [
    "What's the difference?",
    "How does it work?",
    "Why is it important?"
]

def improve_prompt_clarity(unclear_prompt):
    """
    Improve the clarity of a given prompt.
    
    Args:
    unclear_prompt (str): The original unclear prompt
    
    Returns:
    str: An improved, clearer version of the prompt
    """
    improvement_prompt = f"The following prompt is unclear: '{unclear_prompt}'. Please provide a clearer, more specific version of this prompt. output just the improved prompt and nothing else." 
    return llm.invoke(improvement_prompt).content

for prompt in unclear_prompts:
    improved_prompt = improve_prompt_clarity(prompt)
    print(f"Original: {prompt}")
    print(f"Improved: {improved_prompt}")
    print("-" * 50)

Original: What's the difference?
Improved: What are you comparing or contrasting in two things, and what would you like to know about their differences?
--------------------------------------------------
Original: How does it work?
Improved: What is the underlying mechanism or technology behind [specific system, process, or phenomenon]?
--------------------------------------------------
Original: Why is it important?
Improved: What are the specific benefits or consequences of [specific topic or action]?
--------------------------------------------------


# Prompt Length and Complex management

In [67]:
# Detailed prompt
detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition, 
    historical context, key components, practical applications, and any relevant examples. 
    Also, discuss any controversies or debates surrounding the topic, and mention potential 
    future developments or trends."""
)

# Concise prompt
concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance."
)

topic = "artificial intelligence"

print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)

Detailed response:
Artificial Intelligence (AI) is a broad field of computer science that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.

**Definition:**

Artificial Intelligence refers to the development of computer systems that can perform tasks that would normally require human intelligence, such as:

1. Learning: AI systems can learn from data and improve their performance over time.
2. Problem-solving: AI systems can solve complex problems by analyzing data and making decisions.
3. Perception: AI systems can interpret and understand data from sensors and other sources.
4. Reasoning: AI systems can draw conclusions based on the information they have been trained on.

**Historical Context:**

The concept of Artificial Intelligence dates back to the 1950s, when computer scientists like Alan Turing, Marvin Minsky, and John McCarthy began exploring the po

## Analysis of Prompt Balance

In [70]:
analysis_prompt = PromptTemplate(
    input_variables=["topic", "detailed_response", "concise_response"],
    template="""Compare the following two responses on {topic}:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts."""
)

detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(analysis_prompt.format(
    topic=topic,
    detailed_response=detailed_response,
    concise_response=concise_response
)).content

print(analysis)

**Differences in Response:**

1. **Information Coverage:** The detailed response provides a comprehensive overview of AI, covering its definition, historical context, key components, practical applications, examples, controversies, and future developments. In contrast, the concise response focuses on the main importance of AI, highlighting its ability to improve efficiency, enhance accuracy, increase productivity, and drive innovation.
2. **Clarity and Focus:** The detailed response is more structured and organized, with clear headings and subheadings that guide the reader through the content. The concise response is more straightforward and to the point, using simpler language and shorter sentences to convey the main ideas.
3. **Potential Use Cases for Each Type of Response:**
	* Detailed Response:
		+ Suitable for academic or research purposes, where a comprehensive overview is necessary.
		+ Useful for professionals or experts in AI who need to understand the nuances of the field.
	

## Handling Long Context

### Chunking

In [79]:
# [A long passage about artificial intelligence, its history, applications, and future prospects...]
from langchain_text_splitters import RecursiveCharacterTextSplitter
#from langchain.chains.summarize import load_summarize_chain


long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration. 
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as 
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


### Summarization

In [84]:
# Convert to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]


map_prompt = ChatPromptTemplate.from_template(
    "Write a concise summary of the following text:\n\n{text}"
)

map_chain = map_prompt | llm


reduce_prompt = ChatPromptTemplate.from_template(
    """
The following are partial summaries:

{text}

Create a final, well-structured, concise summary.
"""
)

reduce_chain = reduce_prompt | llm



mapped_summaries = []

for doc in doc_chunks:
    result = map_chain.invoke({"text": doc.page_content})
    mapped_summaries.append(result.content)


final_summary = reduce_chain.invoke({
    "text": "\n\n".join(mapped_summaries)
})



print("Summary:\n")
print(final_summary.content)



Summary:

Here is a final, well-structured, concise summary:

Artificial intelligence (AI) aims to create intelligent machines that simulate human cognitive processes, with a rich history dating back to the 1950s. While AI has numerous practical applications across industries, it also raises concerns about job displacement, bias, and ethics. As AI continues to evolve, its future holds promise for advancements in explainable AI, ethics, and collaboration with humans. However, it is essential to consider societal impact and ethical implications to balance innovation with responsibility, ensuring that AI benefits society while minimizing risks. The potential impact of AI on the world will be profound and far-reaching, requiring careful consideration and responsible development to maximize its benefits.


### Iterative Processing

In [87]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.
    
    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.
    
    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:"
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result

analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis."
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
Based on the analysis of the provided text, it appears to be an outline or a framework for discussing various aspects of Artificial Intelligence (AI). The structure suggests that the text is intended to provide a comprehensive overview of AI, covering its definition, history, applications, challenges, future developments, and societal impact.

The text highlights the importance of considering both the benefits and risks associated with AI, emphasizing the need for responsible development, deployment, and use of this technology. It also touches on various areas of research and development that are likely to shape the future of AI.

Some key themes that emerge from the analysis include:

1. The complexity and multifaceted nature of AI, requiring consideration of both technical and societal implications.
2. The need for responsible innovation and deployment of AI, balancing benefits with potential risks and negative consequences.
3. The importance of ongoing research and d

In [ ]:
tips_prompt = """
Based on the examples and strategies we've explored for managing prompt length and complexity, 
provide a list of 5 practical tips for developers working with large language models. 
Each tip should be concise and actionable.
"""

tips = llm.invoke(tips_prompt).content
print(tips)